# Lab : Write a MultiKey Partitioned table to a S3 Datalake using Apache Hudi

## Table of Contents:

1. [Overview](#Overview)
2. [Working with Partitioned Tables](#Working-with-Partitioned-Tables)

## Overview

This notebook demonstrates using PySpark on [Apache Hudi](https://aws.amazon.com/emr/features/hudi/) on Amazon EMR to Write a MultiKey Partitioned table records to an S3 data lake.


This notebook covers the following concepts when writing Copy-On-Write tables to an S3 Datalake:

- Write a MultiKey Partitioned table 



#### Pre-requisites

### This demo is based on Hudi version 0.5.0-incubating and runs fine on Jupyter Notebooks connected to a 1 node (r5.4xlarge) EMR cluster with configuration listed below 

 - EMR versions 5.29.0 or 6.0.0 
 
 - Software configuration

       - Hadoop 2.8.5
       - Hive 2.3.6
       - Livy 0.6.0
       - JupyterHub 1.0.0
       - Spark 2.4.4
       
       
 - AWS Glue Data Catalog settings - Select the below listed check boxes
       - Use for Hive table metadata  
       - Use for Spark table metadata



### Connect to the Master Node of EMR cluster Using SSH :
    - ssh -i ~/xxxx.pem hadoop@<ec2-xx-xxx-xx-xx.us-west-2.compute.amazonaws.com>

    - Ensure  the below listed files are copied into HDFS.

    - hadoop fs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar hdfs:///user/hadoop/

    - hadoop fs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar hdfs:///user/hadoop/

    - hadoop fs -copyFromLocal /usr/lib/spark/jars/httpclient-4.5.9.jar hdfs:///user/hadoop/

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

In [ ]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/httpclient-4.5.9.jar, hdfs:///user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false",     
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "7G",
             "spark.executor.cores": 1,
             "spark.dynamicAllocation.initialExecutors":16
           } 
}

The constants for Python to use:

In [ ]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

Functions to create JSON data and Spark dataframe from this data

In [ ]:
## Generates Data

from datetime import datetime

def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = [{"trip_id": i, "tstamp": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))


## Working with Partitioned Tables

Let's do the same thing with Partitioned Tables. For the sake of this demo, we will be making route_id as partition field. You can also have a nested partition structure like yyyy/mm/dd which is more common

In [ ]:
## CHANGE ME ##
config = {
    "table_name": "hudi_partitioned_trips_table",
    "target": "s3://<Your S3 Bucket Here>/tmp/hudi/hudi_partitioned_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "2",
    "partition_keys" : "route_id"
}

Let's generate the data:

In [ ]:
part_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, part_dest))

We add the partitionKey column to the dataframe.

In [ ]:
from pyspark.sql.functions import concat, col, lit

hudiTablePartitionKey="route_id"
df1 = df1.withColumn(hudiTablePartitionKey,concat(lit("route_id="),col("route_id")))
df1.select(hudiTablePartitionKey).show(5)

And we can now write out the data to S3. Notice that the Hive Partition Extractor class has changed in the statement below:

```
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
```


In [ ]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 6)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
      .mode("Overwrite")
      .save(config['target']))

In [ ]:
spark.sql("show create table "+config['table_name']).show(20,False)

We can see the partitions fields are present in our Hive table. 

```
PARTITIONED BY (`route_id` STRING)
```

Let's now query the data and group by the the partition columns:

In [ ]:
spark.sql("select route_id, count(*) as num_trips from "+config['table_name']+" group by route_id order by route_id").show(20,False)

Let us check the S3 path

```
$ aws s3 ls s3://<Your S3 Bucket Here>/tmp/hudi/hudi_partitioned_trips_table/
                           PRE .hoodie/
                           PRE route_id=A/
                           PRE route_id=B/
                           PRE route_id=C/
                           PRE route_id=D/
                           PRE route_id=E/
                           PRE route_id=F/
                           PRE route_id=G/
                           PRE route_id=H/
                           PRE route_id=I/
                           PRE route_id=J/
2020-04-28 23:42:50          0 .hoodie_$folder$
2020-04-28 23:42:55          0 route_id=A_$folder$
2020-04-28 23:42:55          0 route_id=B_$folder$
2020-04-28 23:42:57          0 route_id=C_$folder$
2020-04-28 23:42:55          0 route_id=D_$folder$
2020-04-28 23:42:57          0 route_id=E_$folder$
2020-04-28 23:42:55          0 route_id=F_$folder$
2020-04-28 23:42:55          0 route_id=G_$folder$
2020-04-28 23:42:57          0 route_id=H_$folder$
2020-04-28 23:42:55          0 route_id=I_$folder$
2020-04-28 23:42:58          0 route_id=J_$folder$

```

Under each partition, there will be partition metadata

```

$ aws s3 ls s3://<Your S3 Bucket Here>/tmp/hudi/hudi_partitioned_trips_table/route_id=A/
2020-04-28 23:42:56         93 .hoodie_partition_metadata
2020-04-28 23:42:59    1723564 447b86e6-500c-463a-bdac-74abb867efad-0_0-256-4156_20200428234249.parquet

```

## The other operations Upsert etc. behave the same way on Partitioned tables.

Try some SQl queries on the partitioned tables

In [ ]:
spark.sql("select destination, route_id from "+config['table_name']+" where route_id = 'B'").show(20,False)

In [ ]:
spark.sql("select destination ,route_id from "+config['table_name']+" ").show(20,False)